In [7]:
import findspark
findspark.init('/home/danielf/spark-3.3.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [2]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

23/02/20 11:31:01 WARN Utils: Your hostname, spark resolves to a loopback address: 127.0.1.1; using 10.0.2.15 instead (on interface enp0s3)
23/02/20 11:31:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/20 11:31:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/20 11:31:06 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
23/02/20 11:31:06 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
23/02/20 11:31:06 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
23/02/20 11:31:06 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
23/02/20 11:31:06 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.
23/02/20 11:31:06 WARN Utils: Service 'SparkUI' could not bind on port 4045. Attempting port 4046.


In [3]:
path = '/home/danielf/Desktop/pyspark-course-udemy/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv'
data = spark.read.csv(path, inferSchema=True, header=True)

In [8]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [14]:
assembler = VectorAssembler(inputCols=[
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent'], outputCol='features')

In [15]:
output = assembler.transform(data)

In [17]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826, 587.9511]))]

In [18]:
final_data = output.select('features', 'Yearly Amount Spent')

In [19]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [20]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [21]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                351|
|   mean| 503.95673540842415|
| stddev|  78.35802566770307|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [22]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                149|
|   mean|  488.3772147713996|
| stddev|  80.73752964797482|
|    min|  282.4712457199145|
|    max|  765.5184619388373|
+-------+-------------------+



In [23]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [24]:
lr_model = lr.fit(train_data)

23/02/20 14:26:53 WARN Instrumentation: [9766b6f0] regParam is zero, which might cause numerical instability and overfitting.


23/02/20 14:26:55 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/02/20 14:26:55 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


23/02/20 14:26:55 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


In [25]:
test_result = lr_model.evaluate(test_data)

In [27]:
test_result.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|7.332801033044234...|
|7.44648787076585E-12|
|4.547473508864641...|
|2.671640686457976...|
|3.581135388230905...|
|-7.95807864051312...|
|6.821210263296962...|
|4.092726157978177...|
|-6.25277607468888...|
|-3.46744855050928...|
|-2.95585778076201...|
|-3.86535248253494...|
|-7.21911419532261...|
|8.29913915367797E-12|
|9.094947017729282...|
|2.444267011014744...|
|6.536993168992922...|
|2.614797267597168...|
|6.366462912410498...|
|5.456968210637569...|
+--------------------+
only showing top 20 rows



In [29]:
test_result.rootMeanSquaredError

5.608882100272533e-12

In [30]:
test_result.r2

1.0

In [31]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [33]:
unlabeled_data = test_data.select('features')
predictions = lr_model.transform(test_data)

In [34]:
predictions.describe().show()

+-------+-------------------+-----------------+
|summary|Yearly Amount Spent|       prediction|
+-------+-------------------+-----------------+
|  count|                149|              149|
|   mean|  488.3772147713996|488.3772147713992|
| stddev|  80.73752964797482|80.73752964797538|
|    min|  282.4712457199145|282.4712457199071|
|    max|  765.5184619388373|765.5184619388373|
+-------+-------------------+-----------------+

